In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import os
import re

In [ ]:
df = pd.DataFrame(pd.read_excel(r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/Input.xlsx'))

In [ ]:
#print first 10 rows
df.sample(10)

In [ ]:
#loop throgh each row
for index, row in df.iterrows():
   url_id = row['URL_ID']
   url = row['URL']


   #giving user access
   headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

   #loading text in url
   response = requests.get(url, headers=headers)

   #parsing url text
   soup=BeautifulSoup(response.content,'html.parser')

   # Find the article title
   for title in soup.find_all('title'):
     Title = title.get_text()

   # Find the article text
   class_= ["td-post-content tagdiv-type", "tdb-block-inner td-fix-index"]
   article = soup.find_all("div", {"class": class_[0]})
   if article:
     article_=""
     for p in article:
       article_ += p.text.strip()
   else:
    article = soup.find_all("div", {"class": class_[1]})
    article_=""
    for p in article:
      article_ += p.text.strip()

   # Write the title and article text to a file
   path = r"/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/Extracted_Data_Text_files"
   file_name = str(url_id) + '.txt'
   #save text file
   with open(os.path.join(path,file_name), 'w') as file:
     file.write(Title + '\n' + article_)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

# Load NLTK resources (you might need to download NLTK resources)
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')

In [ ]:

# Load stop words
stopwords_directory_path = r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/StopWords'
stop_words_set = set()

# Validate directory path
if os.path.isdir(stopwords_directory_path):
    for filename in os.listdir(stopwords_directory_path):
        file_path = os.path.join(stopwords_directory_path, filename)

        # Check if the path is a file
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                # Read each line from the file and add it to the set
                stop_words_set.update(line.strip() for line in file)

# Load text files
text_files_directory_path = r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/Extracted_Data_Text_files'
docs = []

# Validate directory path
if os.path.isdir(text_files_directory_path):
    for filename in os.listdir(text_files_directory_path):
        file_path = os.path.join(text_files_directory_path, filename)

        # Check if the path is a file
        if os.path.isfile(file_path) and filename.endswith('.txt'):
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                # Read the entire content of the file and append it to the list
                text_content = file.read()
                docs.append(text_content)

# Tokenize, remove stop words, and add filtered tokens to a list
filtered_tokens_list = []

for doc in docs:
    # Tokenize the text
    tokens = word_tokenize(doc)

    # Remove stop words
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words_set]

    # Add filtered tokens to the list
    filtered_tokens_list.append(filtered_tokens)

# Display the filtered tokens for each document
for i, filtered_tokens in enumerate(filtered_tokens_list, 1):
    print(f"Filtered Tokens for Document {i}:\n{filtered_tokens}\n")


In [ ]:

# Load positive words
positive_words_directory_path = r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/MasterDictionary_pos'
positive_words = set()

# Validate directory path
if os.path.isdir(positive_words_directory_path):
    for filename in os.listdir(positive_words_directory_path):
        file_path = os.path.join(positive_words_directory_path, filename)

        # Check if the path is a file
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                # Read each line from the file and add it to the set
                positive_words.update(line.strip() for line in file)

# Load negative words
negative_words_directory_path = r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/MasterDictionary_neg'
negative_words = set()

# Validate directory path
if os.path.isdir(negative_words_directory_path):
    for filename in os.listdir(negative_words_directory_path):
        file_path = os.path.join(negative_words_directory_path, filename)

        # Check if the path is a file
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                # Read each line from the file and add it to the set
                negative_words.update(line.strip() for line in file)

# Display the loaded positive and negative words
print("Positive Words:")
print(positive_words)

print("\nNegative Words:")
print(negative_words)

In [ ]:
# lists
Positive_words = []
Negative_words = []
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

# Iterate through the list of docs
for i in range(len(filtered_tokens_list)):
    # Collect positive and negative words
    Positive_words.append([word for word in filtered_tokens_list[i] if word.lower() in positive_words])
    Negative_words.append([word for word in filtered_tokens_list[i] if word.lower() in negative_words])

    # Calculate scores for each document
    positive_score.append(len(Positive_words[i]))
    negative_score.append(len(Negative_words[i]))

    # Calculate polarity and subjectivity scores
    polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(filtered_tokens_list[i])) + 0.000001))

# Display the results
for i in range(len(filtered_tokens_list)):
    print(f"Document {i + 1}:")
    print(f"Positive Words: {Positive_words[i]}")
    print(f"Negative Words: {Negative_words[i]}")
    print(f"Positive Score: {positive_score[i]}")
    print(f"Negative Score: {negative_score[i]}")
    print(f"Polarity Score: {polarity_score[i]}")
    print(f"Subjectivity Score: {subjectivity_score[i]}\n")


In [ ]:
# lists
avg_sentence_length = []
Percentage_of_Complex_words = []
fog_Index_list = []
complex_word_count = []
avg_syllable_word_count = []

stopwords = set(stopwords.words('english'))

# iterate through each file or doc
for file in os.listdir(r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/Extracted_Data_Text_files'):
    with open(os.path.join(r'/content/drive/MyDrive/Data_Extraction_and_Text_Analysis/Extracted_Data_Text_files', file), 'r') as f:
        text = f.read()

    # remove punctuations
    text = re.sub(r'[^\w\s.]', '', text)
    # split the given text file into sentences
    sentences = text.split('.')
    # total number of sentences in a file
    num_sentences = len(sentences)
    # total words in the file
    words = [word for word in text.split() if word.lower() not in stopwords]
    num_words = len(words)

    # complex words having syllable count is greater than 2
    # Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
        vowels = 'aeiou'
        syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
        if syllable_count_word > 2:
            complex_words.append(word)

    # Syllable Count Per Word
    # We count the number of Syllables in each word of the text by counting the vowels present in each word.
    # We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words = []
    for word in words:
        if word.endswith('es'):
            word = word[:-2]
        elif word.endswith('ed'):
            word = word[:-2]
        vowels = 'aeiou'
        syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
        if syllable_count_word >= 1:
            syllable_words.append(word)
            syllable_count += syllable_count_word

    avg_sentence_len = num_words / num_sentences
    Avg_syllable_word_count = (syllable_count / len(syllable_words))
    Percent_Complex_words = len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    avg_sentence_length.append(avg_sentence_len)
    Percentage_of_Complex_words.append(Percent_Complex_words)
    fog_Index_list.append(Fog_Index)
    complex_word_count.append(len(complex_words))
    avg_syllable_word_count.append(Avg_syllable_word_count)


In [ ]:

word_count = []
average_word_length = []
pp_count = []

stop_words_set = set()

# Word Count and Average Word Length
for file in os.listdir(text_files_directory_path):
    with open(os.path.join(text_files_directory_path, file), 'r') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '', text)
        words = [word for word in text.split() if word.lower() not in stopwords]
        length = sum(len(word) for word in words)
        Average_word_length = length / len(words)

    word_count.append(len(words))
    average_word_length.append(Average_word_length)

# Count Personal Pronouns
for file in os.listdir(text_files_directory_path):
    with open(os.path.join(text_files_directory_path, file), 'r') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = 0
        for pronoun in personal_pronouns:
            count += len(re.findall(r"\b" + pronoun + r"\b", text))

    pp_count.append(count)


In [ ]:
# Converting DataFrame to a list containing
# all the rows of column
Url_Id = df['URL_ID'].tolist()
Ur1_links = df['URL'].tolist()

# Printing the converted list.
print(Url_Id)
print(Ur1_links)

In [ ]:

# Appending obtained variables into respective lists
output = {
    "URL_ID":Url_Id,
    "URL_Link":Ur1_links,
    "POSITIVE SCORE": positive_score,
    "NEGATIVE SCORE": negative_score,
    "POLARITY SCORE": polarity_score,
    "SUBJECTIVITY SCORE": subjectivity_score,
    "AVG SENTENCE LENGTH": avg_sentence_length,
    "PERCENTAGE OF COMPLEX WORDS": Percentage_of_Complex_words,
    "FOG INDEX": fog_Index_list,
    "AVG NUMBER OF WORDS PER SENTENCE": avg_sentence_length,
    " COMPLEX WORD COUNT": complex_word_count,
    "WORD COUNT ": word_count,
    "SYLLABLE PER WORD ": avg_syllable_word_count,
    "PERSONAL PRONOUNS": pp_count,
    "AVG WORD LENGTH": average_word_length,
}

output_df = pd.DataFrame(output).set_index("URL_ID")
#now save the dataframe to the disk
output_df.to_csv('Output_Data_Results.csv')

In [ ]:
output_df

,URL_Link,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
URL_ID,,,,,,,,,,,,,,
blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,33,6,0.692308,0.053867,8.287500,0.408748,3.478499,8.287500,271,655,2.353033,6,6.291603
blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,60,31,0.318681,0.090367,10.878049,0.566143,4.577677,10.878049,505,892,2.739331,3,7.216368
blackassign0003,https://insights.blackcoffer.com/internet-dema...,38,24,0.225806,0.079183,11.982456,0.626647,5.043641,11.982456,428,683,2.991071,13,8.060029
blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,38,75,-0.327434,0.145431,13.153846,0.590643,5.497796,13.153846,404,683,2.864179,4,7.775988
blackassign0005,https://insights.blackcoffer.com/ott-platform-...,21,8,0.448276,0.063319,10.341463,0.485849,4.330925,10.341463,206,424,2.643541,6,7.242925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,27,57,-0.357143,0.124260,12.433962,0.484067,5.167212,12.433962,319,659,2.599691,2,7.037936
blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,35,-0.228070,0.104779,13.550000,0.437269,5.594908,13.550000,237,540,2.391791,5,6.479630
blackassign0098,https://insights.blackcoffer.com/contribution-...,6,3,0.333333,0.031142,10.440000,0.524904,4.385962,10.440000,137,261,2.601594,0,7.137931
